# IT Support Dashboard - Supporting Notebook

This notebook contains supporting documentation to the full report. Here, we provide quality checks for the full dataset, confirm whether the English-only sample is representitive of the full population available, and conduct text analytics.

In [8]:
# Importing essential packages
import sys
from pathlib import Path
import pandas as pd

# Add parent directory (project root) to sys.path
project_root = Path(r"C:\Users\David\Desktop\Python_Files\IT-Support-Ticket-Analysis")
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from config import Config

Config.ensure_directories()

# Reading the initial dataset as a DataFrame: df
df = pd.read_csv(
    Config.RAW_DATA_PATH,
    encoding="utf-8",
    engine="python",
    on_bad_lines="skip",
    na_values=["", " ", "NA", "N/A", "na", "n/a", "NULL", None],
)

df  # Display the DataFrame to verify successful import

[Config] Verified project directory structure under C:\Users\David\Desktop\Python_Files\IT-Support-Ticket-Analysis


,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51,Security,Outage,Disruption,Data Breach,NaN,NaN,NaN,NaN
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,Disruption,Outage,IT,Tech Support,NaN,NaN,NaN
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Product,Feature,Tech Support,NaN,NaN,NaN,NaN,NaN
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",We appreciate you reaching out with your billi...,Request,Billing and Payments,low,en,51,Billing,Payment,Account,Documentation,Feedback,NaN,NaN,NaN
4,Question About Marketing Agency Software Compa...,"Dear Support Team,\n\nI hope this message reac...",Thank you for your inquiry. Our product suppor...,Problem,Sales and Pre-Sales,medium,en,51,Product,Feature,Feedback,Tech Support,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28582,Performance Problem with Data Analytics Tool,The data analytics tool experiences sluggish p...,We are addressing the performance issue with t...,Incident,Technical Support,high,en,400,Performance,IT,Tech Support,NaN,NaN,NaN,NaN,NaN
28583,Datensperrung in der Kundschaftsbetreuung,"Es gab einen Datensperrungsunfall, bei dem ung...",Ich kann Ihnen bei dem Datensperrungsunfall he...,Incident,Product Support,high,de,400,Security,IT,Tech Support,Bug,NaN,NaN,NaN,NaN
28584,Problem mit der Videokonferenz-Software heute,Wichtigere Sitzungen wurden unterbrochen durch...,"Sehr geehrte/r [Name], leider wurde das Proble...",Incident,Human Resources,low,de,400,Bug,Performance,Network,IT,Tech Support,NaN,NaN,NaN
28585,Update Request for SaaS Platform Integration F...,Requesting an update on the integration featur...,Received your request for updates on the integ...,Change,IT Support,high,en,400,Feature,IT,Tech Support,NaN,NaN,NaN,NaN,NaN


### Quality checking the dataset

We wanted to see how clean and complete this dataset was. We were looking at the shape of the table, missing values, if dimensions were in appropriate datatypes, how much memory they used, and if any further information could be gathered from descriptive statistics.

In [9]:
# Information about the dataset
print("What is the shape of my table?")
print(df.shape)
print("\nAre there any missing values in each dimension?")
print(df.isna().sum().sort_values())
print("\nWhat is the datatype of each column?")
print(df.dtypes)
print("\nHow many bytes does each column use?")
print(df.memory_usage())

What is the shape of my table?
(28587, 16)

Are there any missing values in each dimension?
body            0
type            0
queue           0
priority        0
language        0
version         0
tag_1           0
answer          7
tag_2          13
tag_3         136
tag_4        3058
subject      3838
tag_5       14042
tag_6       22713
tag_7       26547
tag_8       28022
dtype: int64

What is the datatype of each column?
subject     object
body        object
answer      object
type        object
queue       object
priority    object
language    object
version      int64
tag_1       object
tag_2       object
tag_3       object
tag_4       object
tag_5       object
tag_6       object
tag_7       object
tag_8       object
dtype: object

How many bytes does each column use?
Index          132
subject     228696
body        228696
answer      228696
type        228696
queue       228696
priority    228696
language    228696
version     228696
tag_1       228696
tag_2       228696
tag_

In [ ]:
# Identify rows with null answers; subjects and tags were ignored as they were less critical to this analysis
null_answers = df[df["answer"].isna() == True]
print("\nTable containing rows with null answers:")
display(null_answers)

# Dropping null answers and resetting index
df = df.dropna(subset=["answer"])
df.reset_index(drop=True, inplace=True)
print("\nDataframe after dropping null answers and resetting index:")
display(df)

# Converting data types to save on memory usage
# Lists of data types by column
categories = [
    "type",
    "queue",
    "priority",
    "language",
    "version",
    "tag_1",
    "tag_2",
    "tag_3",
    "tag_4",
    "tag_5",
    "tag_6",
    "tag_7",
    "tag_8",
]
strings = ["subject", "body", "answer"]

# List comprehension loop
for types in df:
    # Categories
    for category in categories:
        if types == category:
            df[types] = df[types].astype("category")

    # Integers
    for string in strings:
        if types == string:
            df[types] = df[types].astype("string")

# Standardizing text in 'priority' and 'language' columns to title case
df["priority"] = df["priority"].str.title()
df["language"] = df["language"].str.title()


Table containing rows with null answers:


,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
4956,Support Inquiry Regarding SendGrid Integration,"Dear Customer Service, I would like to inquire...",NaN,Request,Billing and Payments,high,de,52,Billing,Payment,Platform,Pricing,Discount,Promotion,Integration,Support
5381,Support Request for SendGrid Integration,"Dear Customer Service, I would like to inquire...",NaN,Request,Billing and Payments,high,de,52,Billing,Payment,Support,Integration,Pricing,Promotion,NaN,NaN
12163,Recent Decline in Engagement Metrics Noted Online,We have observed a decrease in engagement metr...,NaN,Problem,Product Support,medium,en,400,Feedback,Performance,Feature,NaN,NaN,NaN,NaN,NaN
13378,Recent Decrease in Engagement Metrics Noted On...,There has been a decline in engagement metrics...,NaN,Problem,Product Support,medium,en,400,Feedback,Performance,Feature,NaN,NaN,NaN,NaN,NaN
13651,NaN,We are sorry to hear that you are experiencing...,NaN,Problem,Technical Support,high,en,400,Bug,Performance,Feature,Documentation,Tech Support,NaN,NaN,NaN
16596,"Sicher, Benutzer melden zeitweise Verbindungsp...","rufen Sie uns an <tel_num>, um über weitere Lö...",NaN,Incident,Customer Service,low,de,400,Network,Performance,Disruption,IT,Tech Support,NaN,NaN,NaN
26522,Reduction in Engagement Metrics Noted Online,Observation of a decline in engagement metrics...,NaN,Problem,Product Support,medium,de,400,Feedback,Performance,Feature,Documentation,NaN,NaN,NaN,NaN



Dataframe after dropping null answers and resetting index:


,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51,Security,Outage,Disruption,Data Breach,NaN,NaN,NaN,NaN
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,Disruption,Outage,IT,Tech Support,NaN,NaN,NaN
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Product,Feature,Tech Support,NaN,NaN,NaN,NaN,NaN
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",We appreciate you reaching out with your billi...,Request,Billing and Payments,low,en,51,Billing,Payment,Account,Documentation,Feedback,NaN,NaN,NaN
4,Question About Marketing Agency Software Compa...,"Dear Support Team,\n\nI hope this message reac...",Thank you for your inquiry. Our product suppor...,Problem,Sales and Pre-Sales,medium,en,51,Product,Feature,Feedback,Tech Support,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28575,Performance Problem with Data Analytics Tool,The data analytics tool experiences sluggish p...,We are addressing the performance issue with t...,Incident,Technical Support,high,en,400,Performance,IT,Tech Support,NaN,NaN,NaN,NaN,NaN
28576,Datensperrung in der Kundschaftsbetreuung,"Es gab einen Datensperrungsunfall, bei dem ung...",Ich kann Ihnen bei dem Datensperrungsunfall he...,Incident,Product Support,high,de,400,Security,IT,Tech Support,Bug,NaN,NaN,NaN,NaN
28577,Problem mit der Videokonferenz-Software heute,Wichtigere Sitzungen wurden unterbrochen durch...,"Sehr geehrte/r [Name], leider wurde das Proble...",Incident,Human Resources,low,de,400,Bug,Performance,Network,IT,Tech Support,NaN,NaN,NaN
28578,Update Request for SaaS Platform Integration F...,Requesting an update on the integration featur...,Received your request for updates on the integ...,Change,IT Support,high,en,400,Feature,IT,Tech Support,NaN,NaN,NaN,NaN,NaN


C:\Users\David\AppData\Local\Temp\ipykernel_17636\3534126513.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[types] = df[types].astype("string")
C:\Users\David\AppData\Local\Temp\ipykernel_17636\3534126513.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[types] = df[types].astype("string")
C:\Users\David\AppData\Local\Temp\ipykernel_17636\3534126513.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [11]:
print("What is the shape of my table?")
print(df.shape)
print("\nAre there any missing values in each dimension?")
print(df.isna().sum().sort_values())
print("\nWhat is the datatype of each column?")
print(df.dtypes)
print("\nHow many bytes does each column use?")
print(df.memory_usage())

What is the shape of my table?
(28580, 16)

Are there any missing values in each dimension?
body            0
answer          0
type            0
queue           0
priority        0
language        0
version         0
tag_1           0
tag_2          13
tag_3         136
tag_4        3056
subject      3837
tag_5       14039
tag_6       22708
tag_7       26541
tag_8       28016
dtype: int64

What is the datatype of each column?
subject     string[python]
body        string[python]
answer      string[python]
type              category
queue             category
priority            object
language            object
version           category
tag_1             category
tag_2             category
tag_3             category
tag_4             category
tag_5             category
tag_6             category
tag_7             category
tag_8             category
dtype: object

How many bytes does each column use?
Index          132
subject     228640
body        228640
answer      228640
type     

Continuing on with the quality checks, we wanted to confirm how many unique values there were under each dimension. This would save time later on by avoiding manual processing of the dataset to identify these findings.

In [12]:
# Loop to extract all unique values from each column in df
for column in df.columns:
    unique_values = df[column].sort_values(ascending=True).unique()
    length = len(unique_values)
    print(f"There were {length} unique values in {column}:\n{unique_values}")
    print("")

There were 24744 unique values in subject:
<StringArray>
[                                                                                                                                                                                              ' Assistance Request',
                                                                                                                                         ' Bitte um Ausführliche Informationen zur Datenaufbereitungsdienstleistung',
                                                                                                                                                                   ' Datenschutzverletzung in Krankenhaus-Systemen ',
                                                                                                                                                                                ' Reported Problem with Data Access',
                                                                                       

In [13]:
# Importing os package for file operations
import os

# ----------------------------------------------------------
# 1. Define output file paths
# ----------------------------------------------------------
# Define file names
csv_path = Config.QC_CSV_DIR / "Tickets_Clean.csv"
parquet_path = Config.QC_CSV_DIR / "Tickets_Clean.parquet"
pickle_path = Config.QC_CSV_DIR / "Tickets_Clean.pkl"

# ----------------------------------------------------------
# Save in each format
# ----------------------------------------------------------
# CSV – portable
df.to_csv(csv_path, index=False, encoding="utf-8")

# Parquet – efficient
df.to_parquet(parquet_path, index=False, engine="pyarrow", compression="snappy")

# Pickle – fast, Python-dependent
df.to_pickle(pickle_path, protocol=5)

# ----------------------------------------------------------
# Print summary of saved files
# ----------------------------------------------------------
for path in [csv_path, parquet_path, pickle_path]:
    size_mb = os.path.getsize(path) / (1024 * 1024)
    print(f"{path.name:<25} | {size_mb:>6.2f} MB")

print("\nAll files saved successfully.")

Tickets_Clean.csv         |  24.79 MB
Tickets_Clean.parquet     |  11.30 MB
Tickets_Clean.pkl         |  23.47 MB

All files saved successfully.
